# This script is brought to by ROAST Labs the internal development and innovation team at [ROAST](https://weareroast.com/) a digital marketing agency.

**Purpose:**

This Python script interfaces with the AccuRanker API to fetch stats for each tag on a account and gather the data in daily intervals. The data is then saved into one csv file.


**How to use**

1.   Add in your [AccuRanker API key on line 5 in the code below](https://app.accuranker.com/app/api?_gl=1*2h0ngs*_ga*MjAxODIxNjAzOS4xNzE1MDkzNjgx*_ga_D2SXNT78V1*MTcxNTA5MzY4MC4xLjEuMTcxNTA5MzY4Mi41OC4wLjE5NjI4MzIyMjE.#section/Introduction).
2.   Run the code
3.   Select a domain by typing a number from the domains listed
4.   Type in a start date you want the report to run from yyyy-mm-dd
5.   Type in a end date you want the report to run from yyyy-mm-dd
5.   The script will run and then save a csv in the sample_data folder.


Please note you run this script at your own risk, ROAST has no liability for the running of this or any other scripts.

In [ ]:
import requests
import csv
import datetime

# Replace with your actual API key
API_KEY = "insert-your-api-key-here"
BASE_URL = "https://app.accuranker.com/api/v4"
HEADERS = {
    "Authorization": f"Token {API_KEY}"
}


def get_accounts():
    """Fetch available accounts."""
    response = requests.get(f"{BASE_URL}/domains/?fields=domain,display_name,id", headers=HEADERS)
    response.raise_for_status()
    return response.json()

def get_tag_data(domain_id, start_date, end_date):
    """Fetch data for a specific tag within a given date range."""
    tag_data = []
    current_date = start_date
    while current_date <= end_date:
        formatted_date = current_date.strftime("%Y-%m-%d")
        response = requests.get(
            f"{BASE_URL}/domains/{domain_id}/tags/?fields=history&period_from={formatted_date}&period_to={formatted_date}",
            headers=HEADERS
        )
        response.raise_for_status()
        tag_data.extend(response.json())
        current_date += datetime.timedelta(days=1)
    return tag_data

def generate_csv(tag_data, file_name):
    """Generate a CSV file from the fetched data."""
    csv_header = [
        "tag", "history/0/date", "history/0/average_rank", "history/0/average_local_pack_rank",
        "history/0/share_of_voice", "history/0/share_of_voice_percentage", "history/0/search_volume",
        "history/0/keywords", "history/0/average_ctr", "history/0/average_ctr_max",
        "history/0/estimated_visits", "history/0/above_the_fold", "history/0/informational_intent",
        "history/0/transactional_intent", "history/0/navigational_intent", "history/0/commercial_intent"
    ]
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(csv_header)
        for tag in tag_data:
            tag_name = tag.get("tag") or "N/A"
            for history in tag["history"]:
                writer.writerow([
                    tag_name,
                    history["date"],
                    history["average_rank"],
                    history["average_local_pack_rank"],
                    history["share_of_voice"],
                    history["share_of_voice_percentage"],
                    history["search_volume"],
                    history["keywords"],
                    history["average_ctr"],
                    history["average_ctr_max"],
                    history["estimated_visits"],
                    history["above_the_fold"],
                    history["informational_intent"],
                    history["transactional_intent"],
                    history["navigational_intent"],
                    history["commercial_intent"]
                ])

# Fetch all available accounts and let the user pick one
accounts = get_accounts()
print("Available Accounts:")
for index, account in enumerate(accounts):
    print(f"{index}: {account['display_name']} ({account['domain']})")
selected_index = int(input("Choose an account index: "))
selected_account = accounts[selected_index]

# Get the date range from the user
start_date_str = input("Enter the start date (YYYY-MM-DD): ")
end_date_str = input("Enter the end date (YYYY-MM-DD): ")
start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")
end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")

# Fetch tag data within the specified date range
tag_data = get_tag_data(selected_account["id"], start_date, end_date)

# Generate a CSV file
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_name = f"/content/sample_data/tag-{timestamp}.csv"
generate_csv(tag_data, file_name)

print(f"CSV file created: {file_name}")
